### 列联表（Contingency Table）

> 来分析两个分类变量之间是否存在关联。一句话总结：卡方检验告诉你“着火了”，而调整后残差告诉你“是厨房的烤箱着火了”。

- 卡方检验 (Chi-Squared Test): 这是第一步，它告诉我们一个总体结论——这两个变量是否相关。如果卡方检验结果显著（比如p值<0.05），我们就可以说：“是的，年龄和社交媒体偏好之间存在关联。”
- 调整后皮尔逊残差 (Adjusted Pearson Residuals): 这是深入分析的第二步。在确定了变量相关后，我们想知道具体是哪些单元格的差异导致了这种相关性。调整后残差就是为每个单元格计算的一个分数，这个分数告诉我们该单元格的实际观测值与预期值之间的差距有多“离谱”。
- 基础数据
    - $O_{ij}$: 观测值（observed）
    - $E_{ij}$：期望值- 如果两个变量完全独立，我们期望这个单元格里应该有的数字。
        - $E_{ij} = \frac{(\text{第 } i \text{ 行的总和}) \times (\text{第 } j \text{ 列的总和})}{\text{总样本数}}$
- 标准皮尔逊残差 (Standard Pearson Residual)
    - 这是最基础的残差，计算的是“观测值与期望值的差距”相对于期望值的大小。
    $$
    \text{Residual} = \frac{O_{ij} - E_{ij}}{\sqrt{E_{ij}}}
    $$
    - 这个值已经能告诉我们差距的方向（正或负）和相对大小，但它有一个小缺点：它的方差不完全等于1，直接用它来做严格的统计推断不太准确。
- 调整后皮尔逊残差 (Adjusted Pearson Residual)
    - 为了解决上述问题，统计学家对它进行了“标准化”调整，让它的分布更接近标准正态分布（一个均值为0，方差为1的分布）。这样我们就可以像使用Z分数（Z-score）一样来解读它。

$$
\text{Adjusted Residual}_{ij} = \frac{O_{ij} - E_{ij}}{\sqrt{E_{ij} \left(1 - \frac{n_{i.}}{N}\right) \left(1 - \frac{n_{.j}}{N}\right)}}
$$

- 如何解读这个值？由于它近似于标准正态分布，我们可以使用一个简单的规则：
    - 如果绝对值 > 1.96，则表明该单元格的差异在统计上是显著的（在p<0.05的水平上）。
        - 在标准正态分布（钟形曲线）中，需要向左、向右走多远（多少个标准差），才能框住中间 95% 的“最可能发生”的区域？
        - 左右的tail部分分别是2.5%
    - 如果绝对值 > 2.58，则差异更加显著（p<0.01）。
    - 正值：表示实际观测到的数量显著高于预期。
    - 负值：表示实际观测到的数量显著低于预期。

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency


# 这是我们的列联表
data = {
    'Facebook': [50, 30, 20],
    'Instagram': [40, 45, 10],
    'TikTok': [10, 25, 70]
}
observed_df = pd.DataFrame(data, index=['18-29岁', '30-44岁', '45岁以上'])
observed_df

,Facebook,Instagram,TikTok
18-29岁,50,40,10
30-44岁,30,45,25
45岁以上,20,10,70


In [2]:
chi2, p_value, dof, expected = chi2_contingency(observed_df)
chi2, p_value

(np.float64(92.34586466165413), np.float64(4.1787188176006126e-19))

- P值小于0.05，结果显著。说明年龄段和社交媒体偏好存在关联。现在我们可以计算调整后残差来找到具体差异点

In [4]:
# 列均值
expected

array([[33.33333333, 31.66666667, 35.        ],
       [33.33333333, 31.66666667, 35.        ],
       [33.33333333, 31.66666667, 35.        ]])

In [14]:
# 获取行总计、列总计和总样本数
row_totals = observed_df.sum(axis=1)
col_totals = observed_df.sum(axis=0)
grand_total = observed_df.sum().sum()
grand_total

np.int64(300)

In [7]:
# 计算调整项的分母
# (1 - 行比例) * (1 - 列比例)
# np.newaxis 用于正确地进行广播运算
row_frac = row_totals / grand_total
col_frac = col_totals / grand_total
# 分母 = sqrt(期望值 * (1-行比例) * (1-列比例))
denominator = np.sqrt(expected * (1 - row_frac.values[:, np.newaxis]) * (1 - col_frac.values))

In [10]:
expected_df = pd.DataFrame(expected, index=observed_df.index, columns=observed_df.columns)
expected_df

,Facebook,Instagram,TikTok
18-29岁,33.333333,31.666667,35.0
30-44岁,33.333333,31.666667,35.0
45岁以上,33.333333,31.666667,35.0


In [12]:
adj_residuals = (observed_df - expected_df) / denominator
adj_residuals

,Facebook,Instagram,TikTok
18-29岁,4.330127,2.194052,-6.419407
30-44岁,-0.866025,3.510483,-2.567763
45岁以上,-3.464102,-5.704535,8.987170


- 我们关注绝对值大于 1.96 的单元格，因为它们代表了统计上的显著差异。

In [17]:
from scipy.stats import norm

# 设定我们的 Z 分数
z_score = 1.96

# 使用生存函数 (sf) 计算从 z_score 到正无穷的面积
# 这就是右边尾部的面积
right_tail_area = norm.sf(z_score)

print(f"Z 分数为 {z_score} 时...")
print(f"右边尾部的面积是: {right_tail_area:.6f}")

Z 分数为 1.96 时...
右边尾部的面积是: 0.024998


In [15]:
for row_label in adj_residuals.index:
    for col_label in adj_residuals.columns:
        val = adj_residuals.loc[row_label, col_label]
        if abs(val) > 1.96:
            if val > 0:
                print(f"- 在'{row_label}'群体中，使用'{col_label}'的人数显著高于预期 (残差: {val:.2f})。")
            else:
                print(f"- 在'{row_label}'群体中，使用'{col_label}'的人数显著低于预期 (残差: {val:.2f})。")

- 在'18-29岁'群体中，使用'Facebook'的人数显著高于预期 (残差: 4.33)。
- 在'18-29岁'群体中，使用'Instagram'的人数显著高于预期 (残差: 2.19)。
- 在'18-29岁'群体中，使用'TikTok'的人数显著低于预期 (残差: -6.42)。
- 在'30-44岁'群体中，使用'Instagram'的人数显著高于预期 (残差: 3.51)。
- 在'30-44岁'群体中，使用'TikTok'的人数显著低于预期 (残差: -2.57)。
- 在'45岁以上'群体中，使用'Facebook'的人数显著低于预期 (残差: -3.46)。
- 在'45岁以上'群体中，使用'Instagram'的人数显著低于预期 (残差: -5.70)。
- 在'45岁以上'群体中，使用'TikTok'的人数显著高于预期 (残差: 8.99)。
